In [1]:
#imports
import numpy as np

# Load Balancing With Contraints Example

**Note: Make sure you've already read Part 1, in which we do this problem without constraints. We'll only be explaining the new bits here.**

In this example we're going to show how you could use various approaches to solve a **constrained** load balancing problem. 

For this problem, we're talking execution times on computer processors, with total execution time on certain processors limited to a certain amount. You might see this happen when one processor needs to "reserve" processing cycles for some other job not in our load balancing list. 

We can describe it as:

given a list of $n$ execution times, divide them to be executed on $k$ processors so that the total execution time on each processor is as close to the same as possible, while $y$ constrained processors are under $x$ execution time limit.

## Hard Constraint
A hard constraint is a constraint which rejects any solution that doesn't meet our specifications. We've seen these before. In Pyomo we used hard constraints. We can use hard constraints with some of our metaheuristics methods. We'll use hard constraints with greedy local search and simulated annealing.

### Move Function
Our move function for the constrained problem does not change at all. It's identical to the unconstrained problem. (If you need a refresher on what the move function is doing, please see the Lesson_05_Load_Balancing notebook.)

In [2]:
# define a move function which changes one processor assignment randomly
def reassign_one(assign,k):
    # pick one of the jobs and assign it to one of k processors
    n = len(assign)
    # choose a job and a new processor assignment
    which_job = np.random.randint(0,n,1)[0]
    which_proc = np.random.randint(0,k,1)[0]
    new_assign = assign.copy()
    new_assign[which_job] = which_proc
    return new_assign


### Hard Constraint - Objective Function
For a hard constraint, our objective function remains identical, too. (If you need a refresher on what the move function is doing, please see the Lesson_05_Load_Balancing notebook.)


In [3]:
# original objective function = total squared deviation of times from balanced times
def balance_metric(assign,times,k):
    target = sum(times)/k
    return sum( (sum(times[assign==j])-target)**2 for j in range(k) )


### Greedy Local Search - Hard Constraint

To implement the hard constraint in our greedy local search, we'll reject any solution that doesn't meet our constraints. 

We'll update the function to take in 2 additional parameters:

* conproc - a list of the processors to constrain
* conmax - a list of the max times on each processor.

Then, after we've made a move, we'll check to see that the new assignments are valid (meet our constraints). If they don't, we'll reject that move and iterate. 

We'll also track whether the algorithm ever finds a solution that meets the constraints. It's possible with a hard constraint that we never find a solution that works.


In [4]:
# local search function
def load_balance_local(times, k, max_no_improve,conproc,conmax):
    n = len(times)
    # starts from a random assignment to k processors
    current_x = np.random.randint(low=0,high=k,size=n)
    current_f = balance_metric(current_x, times, k)
    best_x = current_x
    best_f = current_f
    ##########################
    # New - track convergence
    converged = False
    ##########################
    # stop search if no better x is found within max_no_improve iterations
    num_moves_no_improve = 0
    iterations = 0
    while (num_moves_no_improve < max_no_improve):
        num_moves_no_improve += 1
        iterations += 1  # just for tracking
        new_x = reassign_one(current_x,k)
        new_f = balance_metric(new_x, times, k)
        #print('new_f', new_f)
        #print('Total assigned to constrained processors', [sum(times[new_x==c]) for c in conproc])
        ##################################
        # This is the new bit to deal with constraints
        over_max = True in [sum(times[new_x==c]) > conmax[c] for c in conproc]
        if new_f < current_f and over_max == False:
            converged = True
        #################################      
            num_moves_no_improve = 0
            current_x = new_x
            current_f = new_f
            if current_f < best_f:  
                best_x = current_x  
                best_f = current_f
    return best_x, best_f, iterations, converged

Let's run this with a small number of processors and a small number of job execution times. First let's generate some random data and see what the time on each processor would be if it loads were completely balanced.

In [5]:
# generate random job times
np.random.seed(666) #comment this out to play with new numbers
#we'll start with 20 execution times
n = 30
#we'll start with 2 processors
k = 3
min_time = 20
max_time = 200
times = np.random.randint(low=min_time, high = max_time, size = n)
assign = np.random.randint(low=0,high=k,size=n)
# total time on each processor
print('Total time on each processor, if completely balanced:', sum(times)/k)


Total time on each processor, if completely balanced: 1220.6666666666667


#### Running local search with constraints

Let's start with setting processor 0 to be constrained to a max processing time of 1100. Run this code several times. How often do you get convergence?

In [32]:
best_assign, best_f, num_iter, converged = load_balance_local(times,k,5000,[0],[1100]) #adding our 2 additional parameters here
print('The algorithm found a solution that met the criteria:', converged)
print('The best assignment is', best_assign)
print('Total time on each processor:', [ sum(times[best_assign==j]) for j in range(k)])
print('The deviation from balance is', best_f)
print('It took', num_iter, 'iterations.')

The algorithm found a solution that met the criteria: True
The best assignment is [0 1 2 2 0 1 1 1 1 0 1 2 1 0 1 0 2 0 2 2 0 2 0 1 1 2 2 0 0 2]
Total time on each processor: [1086, 1286, 1290]
The deviation from balance is 27210.666666666664
It took 5053 iterations.


What if we wanted to constrain 2 of our processors? Easy! We just add to our conproc and conmax lists. This time, let's constrain processor 0 to a max time of 1200 and processor 1 to a max time of 1100. Again, run this code multiple times and see how often the algorithm converges.

In [27]:
best_assign, best_f, num_iter, converged = load_balance_local(times,k,5000,[0,1],[1200,1100]) #adding our 2 additional parameters here
print('The algorithm found a solution that met the criteria:', converged)
print('The best assignment is', best_assign)
print('Total time on each processor:', [ sum(times[best_assign==j]) for j in range(k)])
print('The deviation from balance is', best_f)
print('It took', num_iter, 'iterations.')

The algorithm found a solution that met the criteria: False
The best assignment is [2 1 1 0 0 0 2 2 2 2 2 1 1 2 2 0 1 1 0 0 2 2 1 0 2 0 1 0 2 1]
Total time on each processor: [1119, 1172, 1371]
The deviation from balance is 35304.666666666664
It took 5000 iterations.


### Simulated Annealing - By Hand - Hard Constraints

We can take the same hard constraint approach with our hand-coded simulated annealing problem. Once again, we'll add 2 parameters:

* conproc - a list of the processors to constrain
* conmax - a list of the max times on each processor.

And once again we'll pass back a convergence variable to let us know if we ever found a solution that matched our constraints.

We'll use the same set of jobs from the previous example so you can compare. 

In [52]:

def custom_simanneal(times, k, max_no_improve, temp, alpha, conproc, conmax):
    #get the length of our jobs
    n = len(times)
    # starts from a random assignment to k processors
    current_x = np.random.randint(low=0,high=k,size=n)
    current_f = balance_metric(current_x, times, k)
    best_x = current_x
    best_f = current_f
    
    #this is just for tracking
    iterations = 1
    trajectory = [[iterations,current_f]]
    trajectory_best = [[iterations,best_f]]
    ##########################
    # New - track convergence
    converged = False
    ##########################

    # stop search if no better x is found within max_no_improve iterations
    num_moves_no_improve = 0
    while (num_moves_no_improve < max_no_improve):
        num_moves_no_improve += 1
        iterations += 1  # just for tracking
        new_x = reassign_one(current_x,k)
        new_f = balance_metric(new_x, times, k)
        #determine the change in score
        delta = new_f - current_f
        #determine the probability of accepting this solution
        prob = np.exp(min(delta, 0) / temp)
        
        ##################################
        # This is the new bit to deal with constraints
        over_max = True in [sum(times[new_x==c]) > conmax[c] for c in conproc]
        #determine if we'll accept this solution
        accept = (new_f < current_f or np.random.uniform() < prob) and over_max == False
        #################################  
          
        if accept:   
            #################
            #New
            converged = True
            #################
            current_x = new_x
            current_f = new_f
            if current_f < best_f:  
                best_x = current_x  
                best_f = current_f
                num_moves_no_improve = 0
        temp *= alpha
        iterations += 1
        trajectory.append([iterations,current_f])
        trajectory_best.append([iterations,best_f])        
    return best_x, best_f, iterations, trajectory, trajectory_best,converged ####NEW: Return extra variable
    

#######
# New - add the 2 extra parameters
best_x, best_f, iterations, trajectory, trajectory_best, converged = custom_simanneal(times, k, 1000, 500, .99, [0],[1100])

print('The algorithm found a solution that met the criteria:', converged)
print('The best assignment is', best_f)
print('Total time on each processor:', [ sum(times[best_x==j]) for j in range(k)])
print('The deviation from balance is', best_f)

The algorithm found a solution that met the criteria: True
The best assignment is 25904.666666666664
Total time on each processor: [1091, 1267, 1304]
The deviation from balance is 25904.666666666664


### The simanneal Package - Hard Constraints
We can also use a hard constraint with the simanneal package. As a reminder, with simanneal, you don't use external functions. You add your code within the package's move and energy functions. To use a hard constraint in simanneal, you'd enforce the constraint in the **move** function.

We again need our two extra variables:
* conproc - a list of the processors to constrain
* conmax - a list of the max times on each processor.

But this time we'll pass them into the initialization function.


Let's see what that looks like.



In [62]:
#this line just imports the package
from simanneal import Annealer

#this is the line where we decide what we're calling this problem
class loadProblem(Annealer):

    # Here's where we pass extra data if we need it. We need to pass our times (jobs) variable and the number of servers (k)
    def __init__(self, state, times, k, conproc, conmax):
        #this line makes the times accessible within the other two functions
        self.times = times
        self.k = k
        self.conproc = conproc
        self.conmax = conmax
        #this is how we initialize - note we're calling super with the same name as above (loadProblem)
        super(loadProblem, self).__init__(state)  # important!

    def move(self):
        """This corresponds to our previous reassign one function"""
        # pick one of the jobs and assign it to one of k processors
        
        #############################
        #NEW - We have to COPY the state
        assign = self.state.copy()
        n = len(assign)
        k = self.k
        # choose a job and a new processor assignment
        which_job = np.random.randint(0,n,1)[0]
        which_proc = np.random.randint(0,k,1)[0]
        assign[which_job] = which_proc
        
        #################################################
        # New - hard constraint enforcement
        over_max = True in [sum(self.times[assign==c]) > self.conmax[c] for c in self.conproc]
        
        if over_max == False:
            #print('Assign', assign)
            #print('Total time on each processor:', [ sum(self.times[assign==j]) for j in range(k)])
            #we only update the state if we've met our constraints
            self.state = assign
        
    
    def energy(self):
        """This corresponds to our balance_metric function"""
        times = self.times
        assign = self.state
        k = self.k
        target = sum(times)/k
        return sum( (sum(times[assign==j])-target)**2 for j in range(k) )


#initialize the class
assign = np.array([1,2,2,2,1,2,0,1,2,2,1,1,1,0,2,1,1,0,0,1,2,1,0,2,1,1,2,0,2,0])
ld = loadProblem(assign, times, k, [0], [1100])
ld.set_schedule(ld.auto(minutes=.2)) #set approximate time to find results

# since our state is a numpy array, we need deepcopy
ld.copy_strategy = "deepcopy" 
#this is what kicks it off
best_assign, best_score = ld.anneal()



print('The best set is: ', best_assign)
print('Total time on each processor:', [ sum(times[best_assign==j]) for j in range(k)])
print('The best score is:', best_score) 

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
   580.00000      28440.67    55.80%     0.00%     0:00:02     0:00:001 Temperature        Energy    Accept   Improve     Elapsed   Remaining
   580.00000      25032.67    54.69%     0.00%     0:00:13     0:00:001

The best set is:  [1 1 1 1 1 0 0 2 1 2 2 0 2 1 2 1 2 2 0 0 2 2 1 0 1 2 0 2 0 1]
Total time on each processor: [1100, 1281, 1281]
The best score is: 21840.666666666668


**Note:** Simanneal evaluates the problem space before running. If your constraint is set too low, simanneal will print the first pink line, and then just hang. If you're playing with this and it gets stuck, you'll need to restart your kernel and loosen your constraints.

## Soft-Constraints
Soft constraints are implemented in the objective function. Instead of rejecting a solution outright, a penalty is incorporated. For a minimization problem, a positive number is added when the constraint isn't met. For a maximization problem, a negative number is added.

Let's look at what this would look like with a hand-solved problem.

We'll keep our original objective function (balanced_metric), but we'll add a new wrapper function (balanced_metric_constrained). This one will take in 2 additional parameters:
* a list of constrained processors
* a list of the max times on each processor

In [65]:

# constrained objective function = total squared deviation of times from balanced times, providing a penalty for constraints
def balance_metric_constrained(assign,times,k,conproc,conmax):
    #sum the unconstrained processor deviation
    dev_uncon = balance_metric(assign,times,k)
    #sum the constrained processors
    penalty_multiplier = 5
    dev_penalty = penalty_multiplier * sum( max(sum(times[assign==c])-conmax[c],0)**2 for c in conproc )

    return dev_uncon + dev_penalty

### Testing the Soft Constraint

We'll test our two functions with some hand-coded assignments. We'll use 9 jobs on 3 processors. First we'll look at them as an unconstrained, perfectly balanced problem.

In [63]:
#testing perfectly balanced unconstrained
k = 3
times = np.array([2,4,6,2,4,6,2,4,6])
assign=np.array([0,0,0,1,1,1,2,2,2])

# total time on each processor ... should be the same
print('Total time on each processor:', [ sum(times[assign==j]) for j in range(k)])
#print the original balance metric
print('Unconstrained Balance Metric:', balance_metric(assign,times,k))

Total time on each processor: [12, 12, 12]
Unconstrained Balance Metric: 0.0


Now we'll add some constraints. Note that neither our times nor assignments are changing. But, we're essentially changing the target for some of our processors. We're going to set processor 0 to a max limit of 10.

In [66]:
# total time on each processor has not changed
print('Total time on each processor (has not changed):', [ sum(times[assign==j]) for j in range(k)])
#Constrain processor 1 to 10
print('Constrained Balance Metric:', balance_metric_constrained(assign,times,k,[0],[10]))

Total time on each processor (has not changed): [12, 12, 12]
Constrained Balance Metric: 20.0


With the constraint in place, what was a completely balanced solution no longer looks so great. What would happen if we switch our assignments around some?

In [67]:
#new assignments
assign=np.array([1,0,0,1,1,1,2,2,2])
print('Total time on each processor (has changed):', [ sum(times[assign==j]) for j in range(k)])

#check the unconstrained balance metric
print('Balance Metric without constraints', balance_metric(assign,times,k))
#check the constrained balance metric
print('Constrained Balance Metric:', balance_metric_constrained(assign,times,k,[0],[10]))

Total time on each processor (has changed): [10, 14, 12]
Balance Metric without constraints 8.0
Constrained Balance Metric: 8.0


Here, we've met our constraint, so our unconstrained and constrained balance metrics match.

### The simanneal Package - Soft Constraint
With simanneal, instead of adding our hard constraint to the move() function, we'd add our soft constraint to the energy() function.
